### Importing Libraries

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Flatten,Dropout,BatchNormalization,Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from keras.preprocessing.image import load_img,img_to_array

In [2]:
import os
import random
import shutil

# Updated split sizes
train_split = 0.8
valid_split = 0.1  # 10% of the remaining 20% after training split
test_split = 0.1   # The rest to test

categories = []

source_folder = '/Users/arahjou/Downloads/U_NET_GI_Segmentation/data'
folders = os.listdir(source_folder)
print(folders)

for subfolder in folders:
    if os.path.isdir(os.path.join(source_folder, subfolder)):
        categories.append(subfolder)

categories.sort()
print(categories)

target_folder = "/Users/arahjou/Downloads/U_NET_GI_Segmentation/data/dataset_model"
if not os.path.exists(target_folder):
    os.mkdir(target_folder)

trainPath = os.path.join(target_folder, "train")
validatePath = os.path.join(target_folder, "validation")
testPath = os.path.join(target_folder, "test")

if not os.path.exists(trainPath):
    os.mkdir(trainPath)
if not os.path.exists(validatePath):
    os.mkdir(validatePath)
if not os.path.exists(testPath):
    os.mkdir(testPath)

def split_data(SOURCE, TRAINING, VALIDATION, TESTING, TRAIN_SPLIT, VALID_SPLIT):
    files = []
    for filename in os.listdir(SOURCE):
        file = os.path.join(SOURCE, filename)
        if os.path.getsize(file) > 0:
            files.append(filename)
        else:
            print(f"{filename} is 0 length, ignoring it ...")
    print(len(files))

    trainingLength = int(len(files) * TRAIN_SPLIT)
    validationLength = int(len(files) * VALID_SPLIT)

    shuffleSet = random.sample(files, len(files))
    trainingSet = shuffleSet[:trainingLength]
    validationSet = shuffleSet[trainingLength:trainingLength + validationLength]
    testSet = shuffleSet[trainingLength + validationLength:]

    for filename in trainingSet:
        thisFile = os.path.join(SOURCE, filename)
        destination = os.path.join(TRAINING, filename)
        shutil.copyfile(thisFile, destination)

    for filename in validationSet:
        thisFile = os.path.join(SOURCE, filename)
        destination = os.path.join(VALIDATION, filename)
        shutil.copyfile(thisFile, destination)

    for filename in testSet:
        thisFile = os.path.join(SOURCE, filename)
        destination = os.path.join(TESTING, filename)
        shutil.copyfile(thisFile, destination)

for category in categories:
    trainDestPath = os.path.join(trainPath, category)
    validateDestPath = os.path.join(validatePath, category)
    testDestPath = os.path.join(testPath, category)

    if not os.path.exists(trainDestPath):
        os.mkdir(trainDestPath)
    if not os.path.exists(validateDestPath):
        os.mkdir(validateDestPath)
    if not os.path.exists(testDestPath):
        os.mkdir(testDestPath)

    sourcePath = os.path.join(source_folder, category)
    print(f"Copying from: {sourcePath} to: {trainDestPath}, {validateDestPath}, and {testDestPath}")
    split_data(sourcePath, trainDestPath, validateDestPath, testDestPath, train_split, valid_split)


['.DS_Store', 'class_II', 'class_III', 'class_I']
['class_I', 'class_II', 'class_III']
Copying from: /Users/arahjou/Downloads/U_NET_GI_Segmentation/data/class_I to: /Users/arahjou/Downloads/U_NET_GI_Segmentation/data/dataset_model/train/class_I, /Users/arahjou/Downloads/U_NET_GI_Segmentation/data/dataset_model/validation/class_I, and /Users/arahjou/Downloads/U_NET_GI_Segmentation/data/dataset_model/test/class_I
14286
Copying from: /Users/arahjou/Downloads/U_NET_GI_Segmentation/data/class_II to: /Users/arahjou/Downloads/U_NET_GI_Segmentation/data/dataset_model/train/class_II, /Users/arahjou/Downloads/U_NET_GI_Segmentation/data/dataset_model/validation/class_II, and /Users/arahjou/Downloads/U_NET_GI_Segmentation/data/dataset_model/test/class_II
16545
Copying from: /Users/arahjou/Downloads/U_NET_GI_Segmentation/data/class_III to: /Users/arahjou/Downloads/U_NET_GI_Segmentation/data/dataset_model/train/class_III, /Users/arahjou/Downloads/U_NET_GI_Segmentation/data/dataset_model/validation/c

In [6]:
train_directory='/Users/arahjou/Downloads/U_NET_GI_Segmentation/data/dataset_model/train'
val_directory='/Users/arahjou/Downloads/U_NET_GI_Segmentation/data/dataset_model/validation'
test_directory='/Users/arahjou/Downloads/U_NET_GI_Segmentation/data/dataset_model/test'

In [7]:
train_datagen=ImageDataGenerator(rescale=1/255)
val_datagen=ImageDataGenerator(rescale=1/255)
test_datagen=ImageDataGenerator(rescale=1/255)

In [8]:
train_generator=train_datagen.flow_from_directory(train_directory,
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 class_mode='sparse',batch_size=256)

val_generator=val_datagen.flow_from_directory(val_directory,
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 class_mode='sparse',batch_size=256)

test_gemerator=test_datagen.flow_from_directory(test_directory,
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 class_mode='sparse',batch_size=256)

Found 30794 images belonging to 3 classes.
Found 3848 images belonging to 3 classes.
Found 3852 images belonging to 3 classes.


### Classes of body sections

In [9]:
train_generator.class_indices

{'class_I': 0, 'class_II': 1, 'class_III': 2}

### Transfer Learning Using RESNET_101

In [10]:
from keras.applications import ResNet101V2
convlayer=ResNet101V2(input_shape=(224,224,3),weights='imagenet',include_top=False)
for layer in convlayer.layers:
    layer.trainable=False

2024-05-08 22:18:34.121486: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2024-05-08 22:18:34.121508: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2024-05-08 22:18:34.121516: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
2024-05-08 22:18:34.121534: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-05-08 22:18:34.121548: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


171317808/171317808 ━━━━━━━━━━━━━━━━━━━━ 29s 0us/step


In [11]:
model=Sequential()
model.add(convlayer)
model.add(Dropout(0.5))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(2048,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1024,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(225,activation='softmax'))
print(model.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet101v2 (Functional)        │ ?                      │    42,626,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 42,626,560 (162.61 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 42,626,560 (162.61 MB)

None


In [13]:
opt=tf.keras.optimizers.Adam(learning_rate= 0.001)
model.compile(loss='sparse_categorical_crossentropy',metrics=['accuracy'],optimizer=opt)

In [ ]:
history=model.fit(train_generator,validation_data=val_generator,
         epochs=5)

Imbalance

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Flatten, BatchNormalization, Dense, Activation
from tensorflow.keras.applications import ResNet101V2
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Directories
train_directory = '/Users/arahjou/Downloads/U_NET_GI_Segmentation/data/dataset_model/train'
val_directory = '/Users/arahjou/Downloads/U_NET_GI_Segmentation/data/dataset_model/validation'
test_directory = '/Users/arahjou/Downloads/U_NET_GI_Segmentation/data/dataset_model/test'

# Image Data Generators with augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1/255,
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)
val_datagen = ImageDataGenerator(rescale=1/255)
test_datagen = ImageDataGenerator(rescale=1/255)

# Data Generators
train_generator = train_datagen.flow_from_directory(
    train_directory,
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='sparse',
    batch_size=32
)
val_generator = val_datagen.flow_from_directory(
    val_directory,
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='sparse',
    batch_size=32
)
test_generator = test_datagen.flow_from_directory(
    test_directory,
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='sparse',
    batch_size=32
)

# Calculate class weights
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Model architecture using ResNet101V2
convlayer = ResNet101V2(input_shape=(224, 224, 3), weights='imagenet', include_top=False)
for layer in convlayer.layers:
    layer.trainable = False

model = Sequential([
    convlayer,
    Dropout(0.5),
    Flatten(),
    BatchNormalization(),
    Dense(2048, kernel_initializer='he_uniform'),
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.5),
    Dense(1024, kernel_initializer='he_uniform'),
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')  # Assuming 3 classes
])

#print(model.summary())

# Compile the model
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

# Fit the model with class weights
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5,
    class_weight=class_weights_dict
)


Found 30794 images belonging to 3 classes.
Found 3848 images belonging to 3 classes.
Found 3852 images belonging to 3 classes.
Epoch 1/5


/Users/arahjou/Documents/Com_Conda/.conda/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
2024-05-08 22:50:43.285551: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Incompatible shapes: [1,96] vs. [1,32]
	 [[{{function_node __inference_one_step_on_data_66420}}{{node LogicalAnd_1}}]]
2024-05-08 22:50:43.285570: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous send item cancelled. Key hash: 777814623529602584


InvalidArgumentError: Graph execution error:

Detected at node LogicalAnd_1 defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/Users/arahjou/Documents/Com_Conda/.conda/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/Users/arahjou/Documents/Com_Conda/.conda/lib/python3.11/site-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/Users/arahjou/Documents/Com_Conda/.conda/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 701, in start

  File "/Users/arahjou/Documents/Com_Conda/.conda/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 195, in start

  File "/Users/arahjou/Documents/Com_Conda/.conda/lib/python3.11/asyncio/base_events.py", line 608, in run_forever

  File "/Users/arahjou/Documents/Com_Conda/.conda/lib/python3.11/asyncio/base_events.py", line 1936, in _run_once

  File "/Users/arahjou/Docume

In [4]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])  # Temporarily remove precision and recall metrics

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5,
    class_weight=class_weights_dict
)


Epoch 1/5
 28/963 ━━━━━━━━━━━━━━━━━━━━ 9:47 628ms/step - accuracy: 0.3298 - loss: 1.9800

KeyboardInterrupt: 

### Training with lower learning rate for last iteration

In [ ]:
opt=tf.keras.optimizers.RMSprop(lr=0.0001)
model.compile(loss='sparse_categorical_crossentropy',metrics=['accuracy'],optimizer=opt)
history_1=model.fit(train_generator,validation_data=val_generator,
         epochs=1)

### Corrected code

In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Flatten, BatchNormalization, Dense, Activation
from tensorflow.keras.applications import ResNet101V2
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Directories
train_directory = '/Users/arahjou/Downloads/U_NET_GI_Segmentation/data/dataset_model/train'
val_directory = '/Users/arahjou/Downloads/U_NET_GI_Segmentation/data/dataset_model/validation'
test_directory = '/Users/arahjou/Downloads/U_NET_GI_Segmentation/data/dataset_model/test'

# Image Data Generators with augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1/255,
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)
val_datagen = ImageDataGenerator(rescale=1/255)
test_datagen = ImageDataGenerator(rescale=1/255)

# Data Generators
train_generator = train_datagen.flow_from_directory(
    train_directory,
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='sparse',
    batch_size=32
)
val_generator = val_datagen.flow_from_directory(
    val_directory,
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='sparse',
    batch_size=32
)
test_generator = test_datagen.flow_from_directory(
    test_directory,
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='sparse',
    batch_size=32
)

# Calculate class weights
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Model architecture using ResNet101V2
convlayer = ResNet101V2(input_shape=(224, 224, 3), weights='imagenet', include_top=False)
for layer in convlayer.layers:
    layer.trainable = False

model = Sequential([
    convlayer,
    Dropout(0.5),
    Flatten(),
    BatchNormalization(),
    Dense(2048, kernel_initializer='he_uniform'),
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.5),
    Dense(1024, kernel_initializer='he_uniform'),
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')  # Assuming 3 classes
])

# Compile the model with metrics initially removed to test
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy'])

# Fit the model with class weights
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5,
    class_weight=class_weights_dict
)

# Optionally add and test precision and recall after initial tests
precision = tf.keras.metrics.Precision()
recall = tf.keras.metrics.Recall()

for batch, (images, labels) in enumerate(val_generator):
    preds = model.predict(images)
    preds = np.argmax(preds, axis=1)
    precision.update_state(labels, preds)
    recall.update_state(labels, preds)

    if batch >= len(val_generator) - 1:
        break

print(f'Precision: {precision.result().numpy()}, Recall: {recall.result().numpy()}')


Found 30794 images belonging to 3 classes.
Found 3848 images belonging to 3 classes.
Found 3852 images belonging to 3 classes.
Epoch 1/5


/Users/arahjou/Documents/Com_Conda/.conda/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


187/963 ━━━━━━━━━━━━━━━━━━━━ 8:17 641ms/step - accuracy: 0.3430 - loss: 1.7329

### Accuracy Plot

In [ ]:
plt.plot(history.history['accuracy'],c='red')
plt.plot(history.history['val_accuracy'],c='green')
plt.title('Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train','validation'],loc='lower right')

### Loss Plot

In [ ]:
plt.plot(history.history['loss'],c='red')
plt.plot(history.history['val_loss'],c='green')
plt.title('Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train','validation'],loc='upper right')

### Test Accuracy

In [ ]:
model.evaluate(test_gemerator)

### Prediction

Function for prediciting section classes

In [ ]:
dic=train_generator.class_indices
icd={k:v for v,k in dic.items()}
def output(location):
    img=load_img(location,target_size=(224,224,3))
    img=img_to_array(img)
    img=img/255
    img=np.expand_dims(img,[0])
    answer=model.predict_classes(img)
    probability=round(np.max(model.predict_proba(img)*100),2)
    #print ('Bird Is',icd[answer[0]], 'With probability',probability)
    print (probability, ' % chances are there that the Bird Is',icd[answer[0]])

In [ ]:
img='../input/anurag-mishra/belted.jpg'
pic=load_img('../input/anurag-mishra/belted.jpg',target_size=(224,224,3))
plt.imshow(pic)
output(img)

In [ ]:
img='../input/fdjn-vfvjkfd-v/hg.jpg'
pic=load_img('../input/fdjn-vfvjkfd-v/hg.jpg',target_size=(224,224,3))
plt.imshow(pic)
output(img)

In [ ]:
img='../input/fj-fdjvdk-vmdvdjl/bananan.jpg'
pic=load_img('../input/fj-fdjvdk-vmdvdjl/bananan.jpg',target_size=(224,224,3))
plt.imshow(pic)
output(img)